In [276]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options as options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep, time
import selenium.common.exceptions as SEX
import pyautogui
import re

In [61]:
class FirefoxDriver:
    def __init__(self, gecko_driver_path):
        service = Service(executable_path=gecko_driver_path)
        ops = options()
        ops.headless = False
        self.drv = webdriver.Firefox(service=service, options=ops)
        self.drv.implicitly_wait(10)
        self.html = ""

    def _check_xpath(self, xpath):
        def condition(locator):
            p = EC.presence_of_element_located(locator)
            c = EC.element_to_be_clickable(locator)
            v = EC.visibility_of_element_located(locator)
            return p and c and v
        element = WebDriverWait(self.drv, 10).until(
            condition((By.XPATH, xpath)))
        return element

    def _action_click(self, element):
        self.drv.execute_script(
            "arguments[0].scrollIntoView();", element)
        element.click()
    
    def _action_post(self, element, word):
        self.drv.execute_script(
            "arguments[0].scrollIntoView();", element)
        element.click()
        element.send_keys(word)
    
    def get(self, url):
        self.drv.get(url)
        self.html = self.drv.page_source

    def quit(self):
        self.drv.quit()

    def click(self, xpath):
        success = False
        while not success:
            bt = self._check_xpath(xpath)
            try:
                self._action_click(bt)
                success = True
            except SEX.StaleElementReferenceException:
                print('Could not click on', xpath)
                print('Trying again...')
        self.html = self.drv.page_source
    
    def click_element(self, element):
        success = False
        while not success:
            element = self._check_element(element)
            try:
                self._action_click(element)
                success = True
            except SEX.StaleElementReferenceException:
                print('Could not click on', xpath)
                print('Trying again...')
        self.html = self.drv.page_source

    def post(self, xpath, word):
        success = False
        while not success:
            field = self._check_xpath(xpath)
            try:
                self._action_post(field, word)
                success = True
            except SEX.StaleElementReferenceException:
                print('Could not send', word, 'to', xpath)
                print('Trying again...')
        self.html = self.drv.page_source

    def upload(self, xpath, file):
        success = False
        while not success: 
            field = self._check_xpath(xpath)
            try:
                self._action_click(field)
                success = True
            except SEX.StaleElementReferenceException:
                print('Could not send', file, 'to', xpath)
                print('Trying again...')
        pyautogui.sleep(3)
        pyautogui.typewrite(file + '\n')
        pyautogui.sleep(3)
        pyautogui.press('enter')
        self.html = self.drv.page_source

    def clear(self, xpath):
        field = self._check_xpath(xpath)
        self.drv.execute_script("arguments[0].scrollIntoView();",
                field)
        ActionChains(self.drv)\
            .click(field)\
            .key_down(Keys.COMMAND)\
            .send_keys(Keys.UP)\
            .key_up(Keys.COMMAND)\
            .perform()
        self.html = self.drv.page_source

    def server(self, xpath):
        phs = PhoneServer()
        code = phs.start()
        self.post(xpath, code)

In [272]:
driver = FirefoxDriver("/Users/TheDudeAK/Downloads/geckodriver")

In [273]:
driver.get("https://ca.indeed.com/")

In [127]:
elements = driver.drv.find_elements(By.XPATH, "//*[@class=\"css-5lfssm eu4oa1w0\"]")

In [128]:
len(elements)

18

In [149]:
pages = []
for i in range(0,50):
    pages.append(driver.html)
    driver.click("/html/body/main/div/div[2]/div/div[5]/div/div[1]/nav/ul/li[6]/a")

TimeoutException: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:192:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:510:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16


In [151]:
len(pages)

36

In [152]:
from bs4 import BeautifulSoup

In [154]:
soup = BeautifulSoup(pages[0], 'html.parser')
print(soup.title)

<title>2,515 New Computer Science Job Postings in Toronto, ON | 30 March 2024 Indeed.com</title>


In [160]:
links = []
for page in pages:
    soup = BeautifulSoup(page, 'html.parser')
    for link in soup.find_all('a', class_="jcs-JobTitle"):
        links.append(link.get('href'))

In [161]:
len(links)

537

In [251]:
driver.get("https://ca.indeed.com" + links[4])

In [253]:
soup = BeautifulSoup(driver.html, 'html.parser')

In [237]:
soup.title

<title>Agile Lead - Toronto, ON - Indeed.com</title>

In [246]:
soup.find('h1', class_="jobsearch-JobInfoHeader-title").text
    

'Business Systems Consultant/Analyst (SAP)'

In [247]:
try:
    print(soup.find('div', {'data-testid':"job-location"}).text)
except:
    print(soup.find('div', {'data-testid':"inlineHeader-companyLocation"}).text)

893 Yonge St, Toronto, ON


In [256]:
salary_and_job_container = soup.find('div', {'id':'salaryInfoAndJobType'})
for piece in salary_and_job_container.find_all('span'):
    if "$" in piece.text:
        pay = piece.text
    else:
        job_type = piece.text
print(pay,job_type)

$45,000–$55,000 a year  -  Full-time


In [250]:
soup.find('div', {'id':'jobDescriptionText'}).text

"Job Description \n\n As a Business Systems Consultant, individuals will be responsible for implementing and supporting SAP Enterprise Applications for both large enterprises and midsized companies. Successful candidates will be provided with the opportunity to develop skills in system design, execution and support, while working towards more prominent client facing opportunities for both the implementation and support of SAP projects. \n\n Position Type: Permanent - Full Time \nLocation: Toronto, Canada \nCurrent Work Environment: Completely Remote \nExperience Required: Entry level, Recent Graduate, 1 to 3 years (in related field). \n\n Our clients include some of the most prestigious and well-known companies in the world. With clients in North and South America, Europe, and Asia, CONTAX is aggressively expanding our geographical scope and presence in the SAP marketplace. \n\n Our rapid growth and expansion create challenging, rewarding, and interesting opportunities for career-minde

In [327]:

job = {}
driver.get("https://ca.indeed.com" + links[500])
soup = BeautifulSoup(driver.html, 'html.parser')
# job title
title_container = soup.find('h1', class_="jobsearch-JobInfoHeader-title") 
if title_container is not None:
    job["Job title"] = title_container.text
# location
long_location_container = soup.find('div', {'data-testid':"job-location"})
if long_location_container is not None:
    job["Location"] = long_location_container.text
else:
    short_location_container = soup.find('div', {'data-testid':"inlineHeader-companyLocation"})
    if short_location_container is not None:
        job["Location"] = short_location_container.text
# salary and job type
salary_and_job_container = soup.find('div', {'id':'salaryInfoAndJobType'})
if salary_and_job_container is not None:
    for piece in salary_and_job_container.find_all('span'):
        if "$" in piece.text:
            job["Salary"] = piece.text
        else:
            job["Job type"] = piece.text
# description
description_container = soup.find('div', {'id':'jobDescriptionText'})
if description_container is not None:
    job["Description"] = description_container.text
print(job)

{'Job title': 'Lead Data Engineer', 'Location': '27 Fasken Dr, Toronto, ON', 'Job type': 'Permanent, Full-time', 'Description': 'About Us:\n Sunwing Vacations Group is home to North America’s largest vacation brands, comprised of the leading Canadian leisure tour operators Sunwing Vacations and WestJet Vacations, along with the growing United States based tour operator Vacation Express, together with the Group’s own travel retail businesses SellOffVacations.com and Luxe Destination Weddings. Our culture is driven by our EPIC Values (entrepreneurial, passionate, innovative, and customer-centric) and our teams work together to deliver our purpose “passionate people making vacation dreams come true”. \n\n The Opportunity:\n Under Sunwing Vacations, we are looking for a Lead, Data Engineer to join us in making vacation dreams come true. As the Lead, Data Engineer you will be responsible for leading the team, design, configuration, development, and delivery of the Business Intelligence solu

In [277]:
data = []
for link in links:
    job = {}
    driver.get("https://ca.indeed.com" + link)
    soup = BeautifulSoup(driver.html, 'html.parser')
    # job title
    title_container = soup.find('h1', class_="jobsearch-JobInfoHeader-title") 
    if title_container is not None:
        job["Job title"] = title_container.text
    # location
    long_location_container = soup.find('div', {'data-testid':"job-location"})
    if long_location_container is not None:
        job["Location"] = long_location_container.text
    else:
        short_location_container = soup.find('div', {'data-testid':"inlineHeader-companyLocation"})
        if short_location_container is not None:
            job["Location"] = short_location_container.text
    # salary and job type
    salary_and_job_container = soup.find('div', {'id':'salaryInfoAndJobType'})
    if salary_and_job_container is not None:
        for piece in salary_and_job_container.find_all('span'):
            if "$" in piece.text:
                job["Salary"] = piece.text
            else:
                job["Job type"] = piece.text
    # description
    description_container = soup.find('div', {'id':'jobDescriptionText'})
    if description_container is not None:
        job["Description"] = description_container.text
    data.append(job)
    sleep(10)

In [282]:
len(data)

537

In [281]:
import json
with open('my_list.json', 'w') as json_file:
    json.dump(data, json_file)

In [296]:
for i in range(0, 537):
    data[i]["url"] = "https://ca.indeed.com" + links[i]

In [297]:
data[536]["url"]

'https://ca.indeed.com/rc/clk?jk=02e0adc284bfaa7d&bb=_nKwAQXJfiyBjmgS3-ipUqVNcY0z1_Pz5o9DdreT6VUApNJRgnQreM76iGiN3oHIucGVeBVI-9QrLURGJsFR09PmEuqmu5HjkqE7C0Cxnzsk2g-3P3YPdA%3D%3D&xkcb=SoBl67M3CIir7yxylh0AbzkdCdPP&fccid=18fd9e960603ee34&vjs=3'

In [303]:
job_types = []
for i in range(0,537):
    if 'Job type' in data[i] and data[i]['Job type'] not in job_types:
        job_types.append(data[i]['Job type'])

In [304]:
job_types

['Permanent, Full-time',
 'Full-time',
 'Internship / Co-op',
 ' -  Temporary, Full-time, Fixed term contract',
 ' -  Temporary, Part-time, Full-time, Fixed term contract, Internship / Co-op',
 'Permanent',
 'Part-time',
 'Contract, Fixed term contract',
 ' -  Temporary, Fixed term contract',
 'Full-time, Contract, Fixed term contract',
 ' -  Full-time, Fixed term contract',
 ' -  Temporary, Seasonal, Full-time',
 'Full-time, Internship / Co-op',
 ' -  Part-time',
 'Temporary, Full-time',
 'Temporary, Full-time, Fixed term contract',
 ' -  Permanent, Full-time, Freelance',
 ' -  Temporary',
 ' -  Part-time, Contract, Fixed term contract',
 ' -  Part-time, Contract',
 ' -  Contract, Fixed term contract',
 ' -  Permanent',
 ' -  Part-time, Casual',
 'Fixed term contract',
 'Temporary, Permanent, Part-time, Full-time, Fixed term contract, Freelance',
 'Temporary',
 ' -  Part-time, Casual, Freelance']

In [307]:
for i in range(0,537):
    if 'Job type' in data[i] and data[i]['Job type'] == ' -  Temporary, Full-time, Fixed term contract':
        data[i]['Job type'] = 'Temporary, Full-time, Fixed term contract'
    if 'Job type' in data[i] and data[i]['Job type'] == ' -  Temporary, Part-time, Full-time, Fixed term contract, Internship / Co-op':
        data[i]['Job type'] = 'Temporary, Part-time, Full-time, Fixed term contract, Internship / Co-op'
    if 'Job type' in data[i] and data[i]['Job type'] == ' -  Temporary, Fixed term contract':
        data[i]['Job type'] = 'Temporary, Fixed term contract'
    if 'Job type' in data[i] and data[i]['Job type'] == ' -  Full-time, Fixed term contract':
        data[i]['Job type'] = "Full-time, Fixed term contract"
    if 'Job type' in data[i] and data[i]['Job type'] ==' -  Temporary, Seasonal, Full-time':
        data[i]['Job type'] = 'Temporary, Seasonal, Full-time'
    if 'Job type' in data[i] and data[i]['Job type'] ==' -  Part-time':
        data[i]['Job type'] = 'Part-time'
    if 'Job type' in data[i] and data[i]['Job type'] ==' -  Permanent, Full-time, Freelance':
        data[i]['Job type'] = 'Permanent, Full-time, Freelance'
    if 'Job type' in data[i] and data[i]['Job type'] ==' -  Temporary':
        data[i]['Job type'] ='Temporary'
    if 'Job type' in data[i] and data[i]['Job type'] == ' -  Part-time, Contract, Fixed term contract':
        data[i]['Job type'] = 'Part-time, Contract, Fixed term contract'
    if 'Job type' in data[i] and data[i]['Job type'] ==' -  Contract, Fixed term contract':
        data[i]['Job type'] = 'Contract, Fixed term contract'
    if 'Job type' in data[i] and data[i]['Job type'] ==' -  Permanent':
        data[i]['Job type'] = 'Permanent'
    if 'Job type' in data[i] and data[i]['Job type'] == ' -  Part-time, Casual':
        data[i]['Job type'] = 'Part-time, Casual'
    if 'Job type' in data[i] and data[i]['Job type'] == ' -  Part-time, Casual, Freelance':
        data[i]['Job type'] = 'Part-time, Casual, Freelance'
    if 'Job type' in data[i] and data[i]['Job type'] ==  ' -  Part-time, Contract':
        data[i]['Job type'] = 'Part-time, Contract'

In [316]:
job_types = []
for i in range(0,537):
    if 'Job type' in data[i] and data[i]['Job type'] not in job_types:
        job_types.append(data[i]['Job type'])
job_types

['Permanent, Full-time',
 'Full-time',
 'Internship / Co-op',
 'Temporary, Full-time, Fixed term contract',
 'Temporary, Part-time, Full-time, Fixed term contract, Internship / Co-op',
 'Permanent',
 'Part-time',
 'Contract, Fixed term contract',
 'Temporary, Fixed term contract',
 'Full-time, Contract, Fixed term contract',
 'Full-time, Fixed term contract',
 'Temporary, Seasonal, Full-time',
 'Full-time, Internship / Co-op',
 'Temporary, Full-time',
 'Permanent, Full-time, Freelance',
 'Temporary',
 'Part-time, Contract, Fixed term contract',
 'Part-time, Contract',
 'Part-time, Casual',
 'Fixed term contract',
 'Temporary, Permanent, Part-time, Full-time, Fixed term contract, Freelance',
 'Part-time, Casual, Freelance']

In [324]:
data[70]

{'Job title': 'Intermediate Quality Assurance Engineer',
 'Location': '2101 Hadwen Road, Mississauga, ON',
 'Salary': '$50–$60 an hour',
 'Job type': 'Temporary, Fixed term contract',
 'Description': "Duties & Responsibilities:\n\nDevelop and execute detailed manual test cases to ensure product quality adheres to predefined standards.\nDesign, Develop and execute automated test scripts using Flutter, Cypress, and other relevant automation tools.\nCommunicate test progress, test results, and other relevant information to project stakeholders and management.\nImplement and refine testing frameworks and strategies to streamline the QA process and enhance efficiency.\nConduct thorough regression testing to identify potential issues and guarantee product stability after each release.\nPrepare detailed test plans, test cases, and test reports to effectively communicate testing progress and results to stakeholders.\nActively contribute to the continuous improvement of QA processes and methodo

In [311]:
with open('my_list.json', 'w') as json_file:
    json.dump(data, json_file)

In [326]:
job_titles = []
for i in range(0,537):
    if 'Job title' in data[i] and data[i]['Job title'] not in job_types:
        job_titles.append(data[i]['Job title'])
len(job_titles)

536

In [328]:
job_titles

['ServiceNow Developer - ITOM',
 'Full Stack Developer',
 'Senior Business Analyst',
 'Statistical/ Data Analyst',
 'Junior Data Analyst',
 'Data Analyst',
 'Research Associate Intern (Data Science/Data Analytics)',
 'IT Analyst',
 'Intern, Research Analyst (2024)',
 'Business and Data Analyst',
 'Student Developer',
 'QA Analyst',
 'Software Engineer I, New Grad',
 'IT Project Manager – Transit Systems Implementation',
 'Helpdesk Support – Level 1 & 2',
 'Desktop Support Analyst',
 'Business Analyst',
 'Solutions Architect- Elk Stack/Snowflake, GFT',
 'Research Analyst',
 'Specialist, IT Support',
 'Software Analyst, Vehicle Solutions',
 'game developer, computer',
 'IT Specialist',
 'SQL Developer / Database Administrator',
 'Software Developer, New Grad',
 'Software Engineer Intern - (Co-Op) - Summer 2024',
 'Data scientist',
 'Software Developer Summer Co-op (Tech Start-up/ Social Health)',
 'Embedded Software Developer – Network Management',
 'Junior Developer',
 'Software Develop